## Datashader dashboard

This notebook contains the code for an interactive dashboard for making [Datashader](http://datashader.org) plots from any dataset. Apart from Datashader itself, the code relies on other Python packages from the [PyViz](http://pyviz.org) project that are each designed to make it simple to:

- lay out plots and widgets into an app or dashboard, in a notebook or for serving separately ([Panel](http://panel.pyviz.org))
- build interactive web-based plots without writing JavaScript ([Bokeh](http://bokeh.pydata.org))
- build interactive Bokeh-based plots backed by Datashader, from concise declarations ([HoloViews](http://holoviews.org))
- express dependencies between parameters and code to build reactive interfaces declaratively ([Param](http://param.pyviz.org))
- describe the fields and plotting information needed to plot a dataset, in a text file ([Intake](http://intake.readthedocs.io))

In [ ]:
import os, colorcet, param as pm, holoviews as hv, panel as pn
import intake, geoviews.tile_sources as gts
from holoviews.operation.datashader import rasterize, shade, spread

hv.extension('bokeh', logo=False)

You can run the dashboard here in the notebook with various datasets by editing the `dataset` below to specify some dataset defined in `dashboard.yml`.  You can also launch a separate, standalone server process in a new browser tab with a command like:

```
DS_DATASET=nyc_taxi panel serve --show dashboard.ipynb
DS_DATASET=census   panel serve --show dashboard.ipynb
DS_DATASET=opensky  panel serve --show dashboard.ipynb
DS_DATASET=osm-1b   panel serve --show dashboard.ipynb
```

To launch multiple dashboards at once, you'll need to add `-p 5001` (etc.) to select a unique port number for the web page to use for communicating with the Bokeh server.  Otherwise, be sure to kill the server process before launching another instance.

For most of these datasets, if you have less than 16GB of RAM on your machine, you should remove the `.persist()` call below, to tell [Dask](http://dask.pydata.org) to work out of core instead of loading all data into memory.  However, doing so will make interactive use substantially slower than if sufficient memory were available.

In [ ]:
# nyc_taxi, census, opensky, osm-1b
dataset = os.getenv("DS_DATASET", "nyc_taxi")
catalog = intake.open_catalog('dashboard.yml')
source  = getattr(catalog, dataset)
source.to_dask().persist();

The Intake `source` object lets us treat data in many different formats the same in the rest of the code here. We can now build a class that captures some parameters that the user can vary along with how those parameters relate to the code needed to update the displayed plot of that data source:

In [ ]:
gopts = hv.opts.WMTS(width=800, height=500, xaxis=None, yaxis=None, bgcolor='black', show_grid=False)
cmaps = {n: colorcet.palette[n] for n in ['fire', 'bgy', 'bgyw', 'bmy', 'gray', 'kbc']}
norms = {'Histogram_Equalization': 'eq_hist', 'Linear': 'linear', 'Log': 'log', 'Cube root': 'cbrt'}
maps  = ['CartoMidnight', 'StamenWatercolor', 'StamenTonerBackground', 'EsriImagery', 'EsriUSATopo', 'EsriTerrain']
bases = {name: ts.relabel(name) for name, ts in gts.tile_sources.items() if name in maps}
plots = {source.metadata['plots'][p].get('label', p):p for p in source.plots}
logo  = "https://raw.githubusercontent.com/pyviz/datashader/master/doc/_static/logo_horizontal_s.png"

class Explorer(pm.Parameterized):
    plot          = pm.ObjectSelector( precedence=0.10, default=source.plots[0], objects=plots)
    normalization = pm.ObjectSelector( precedence=0.12, default='eq_hist',       objects=norms)
    cmap          = pm.ObjectSelector( precedence=0.14, default=cmaps['fire'],   objects=cmaps)
    spreading     = pm.Integer(        precedence=0.16, default=0, bounds=(0, 5))
    
    basemap       = pm.ObjectSelector( precedence=0.18, default=bases['EsriImagery'], objects=bases)
    data_opacity  = pm.Magnitude(      precedence=0.20, default=1.00, doc="Alpha value for the data")
    map_opacity   = pm.Magnitude(      precedence=0.22, default=0.75, doc="Alpha value for the map")
    show_labels   = pm.Boolean(        precedence=0.24, default=True)
    
    @pm.depends('plot')
    def points(self):
        return getattr(source.plot, self.plot)() # could add extra hvplot kwargs here

    @pm.depends('map_opacity','basemap')
    def tiles(self):
        return self.basemap.opts(gopts).opts(alpha=self.map_opacity)

    @pm.depends('show_labels')
    def labels(self):
        return gts.StamenLabels.options(level='annotation', alpha=1 if self.show_labels else 0)
    
    @pm.depends('data_opacity')
    def apply_opacity(self, shaded):
        return shaded.opts(alpha=self.data_opacity, show_legend=False)

    def view(self,**kwargs):
        points   = hv.DynamicMap(self.points)
        agg      = rasterize(points, width=600, height=400)
        shade_s  = hv.streams.Params(self, ['cmap', 'normalization'])
        spread_s = hv.streams.Params(self, ['spreading'], rename={'spreading': 'px'})
        shaded   = spread(shade(agg, streams=[shade_s]), streams=[spread_s], how="add")
        shaded   = hv.util.Dynamic(shaded, operation=self.apply_opacity)
        return hv.DynamicMap(self.tiles) * shaded * hv.DynamicMap(self.labels)

explorer = Explorer(name="")
pn.Row(pn.Column(logo, pn.Param(explorer.param, expand_button=False)), explorer.view()).servable()

If you are viewing this notebook with a live Python server process running, adjusting one of the widgets above should update the plot, re-running only the code needed to update that particular item without re-running datashader if that's not needed. It should work the same when launched as a separate server process, but without the extra text and code visible as in this notebook. Here the `.servable()` method call indicates what should be served when run separately, and you can just copy the code out of this notebook into a .py file that will work the same as this .ipynb file.